#### 1. Write a Python metaclass "AttrLoggingMeta" that logs every time an attribute is accessed or modified.

In [2]:
class LoggingMeta(type):
    def __new__(cls, name, bases, dct):
        original_getattribute = dct.get('__getattribute__', object.__getattribute__)
        def logging_getattribute(self, name):
            print(f"Accessing attribute: {name}")
            return original_getattribute(self, name)

        original_setattr = dct.get('__setattr__', object.__setattr__)
        def logging_setattr(self, name, value):
            print(f"Setting attribute: {name} to {value}")
            original_setattr(self, name, value)

        dct['__getattribute__'] = logging_getattribute
        dct['__setattr__'] = logging_setattr
        return super().__new__(cls, name, bases, dct)

class LoggedClass(metaclass=LoggingMeta):
    def __init__(self, value):
        self.value = value

obj = LoggedClass(42)

print(obj.value)
obj.value = 100
obj.new_attr = "Hello"
print(obj.new_attr)

Setting attribute: value to 42
Accessing attribute: value
42
Setting attribute: value to 100
Setting attribute: new_attr to Hello
Accessing attribute: new_attr
Hello


In [4]:
# Do following for better results and accurate answer

class AttrLoggingMeta(type):
    def __new__(cls, name, bases, dct):
        for k, v in dct.items():
            if not k.startswith("__"):
                dct[k] = cls.log_access(k, v)
        return super().__new__(cls, name, bases, dct)

    def log_access(name, value):
        if callable(value):
            def wrapper(*args, **kwargs):
                print(f"Calling method: {name}")
                return value(*args, **kwargs)
            return wrapper
        else:
            return property(
                fget = lambda self: AttrLoggingMeta.log_read(name, value, self),
                fset = lambda self, v: AttrLoggingMeta.log_write(name, v, self)
            )

    def log_read(name, value, self):
        print(f"Attribute {name}'s value accessed")
        return value

    def log_write(name, v, self):
        print(f"Attribute {name}'s value set to {v}")
        self.__dict__[name] = v

class LoggedClass(metaclass=AttrLoggingMeta):
    random_val = 21
    def random_func(self):
        return "random string"

l1 = LoggedClass()
print(l1.random_val)
l1.random_val = 50
l1.random_func()

Attribute random_val's value accessed
21
Attribute random_val's value set to 50
Calling method: random_func


'random string'

#### 2. Write a Python function "create_inherited_class" that takes a base class, a class name, and a dictionary of additional attributes and methods, and returns a dynamically created subclass.

In [7]:
class BaseClass:
    def greet(self):
        return "Hello! How are you?"

def create_inherited_class(base_class, child_class, attrs):
    return type(child_class, (base_class,), attrs)

attrs = {
    "additional_method": lambda self: "Additional Method"
}

DynamicSubClass = create_inherited_class(BaseClass, "DynamicSubClass", attrs)

instance = DynamicSubClass()
print(instance.greet())
print(instance.additional_method())
print(type(instance))

Hello! How are you?
Additional Method
<class '__main__.DynamicSubClass'>


#### 3. Write a Python function "generate_complex_function" that takes a function name, a list of parameter names, and a body as strings, and returns a dynamically generated function.

In [12]:
def generate_complex_function(func_name, params, body):
    body = body.replace("\n", "\n    ")
    exec(f"def {func_name}({",".join(params)}):\n    {body}")
    return locals()[func_name]

func_name = "greater"
func_params = ["a","b"]
func_body = """
if a>b:
    return a
else:
    return b
"""

add_func = generate_complex_function(func_name, func_params, func_body)
print(add_func(1,2))
print(add_func(5,2))

2
5
